In [1]:
# Import libraries
import psycopg2
import getpass
import numpy as np
import os.path

In [2]:
# Create a database connection
user = 'postgres'
host = 'localhost'
port = '5433'
dbname = 'mimic'
schema = 'mimiciii'

In [3]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, port=port, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path TO {}'.format(schema))
con.commit()
print("Connected!")

Password: ··········


Connected!


In [4]:
# Select admission IDs from MIMIC III that correspond to first time admissions, for patient 18 years old or older
query = \
"""
WITH full_patient_info AS
    (
    SELECT a.subject_id, a.hadm_id, p.gender,
        ROUND((CAST(a.admittime AS date) - CAST(p.dob AS date))/365.242,2) AS age,
        p.dob, a.admittime, a.dischtime,
        a.hospital_expire_flag,
        ROUND(CAST(EXTRACT(epoch FROM (CAST(a.dischtime AS timestamp) - CAST(a.admittime AS timestamp)))/3600 AS numeric) ,2) AS los,
        MIN(a.admittime) OVER (PARTITION BY a.subject_id) AS first_admittime
    FROM admissions a
    INNER JOIN patients p
    ON p.subject_id = a.subject_id
    )
SELECT subject_id, hadm_id, age::float, los::float, hospital_expire_flag,
    CAST((admittime = first_admittime) AS int) AS first_admit_flag, admittime
FROM full_patient_info
"""

# Execute the query and fetch the result
cur.execute(query)
con.commit()
data = cur.fetchall()

# Store the result of the query as a numpy array
all_adm = np.array(data)

In [5]:
# Check the number of patients
adm_labels = ['subject_id', 'hadm_id', 'age', 'los', 'hospital_expire_flag', 'first_admit_flag', 'admittime']
print("No. of patients: {}".format(np.shape(all_adm)[0]))

No. of patients: 58976


In [6]:
# Check the number of patients on their first admission
first_adm = all_adm[all_adm[:,5]==1]
print("No. of patients on first admission: {}".format(np.shape(first_adm)[0]))

No. of patients on first admission: 46520


In [7]:
# Check the number of adult patients on their first admission
adult_first_adm = first_adm[first_adm[:,2]>=18]
print("No. of adult patients on first admission: {}".format(np.shape(adult_first_adm)[0]))

No. of adult patients on first admission: 38549


In [8]:
# Save to admission_ids.npy
if not os.path.exists('./res'):
    os.makedirs('./res')

tosave = {'adm_labels':adm_labels, 'all_adm':all_adm, 'first_adm':first_adm, 'adult_first_adm':adult_first_adm}
np.save('res/admission.npy',tosave)
print("Saved!")

Saved!
